# Gradient descent from scratch - part 1, simple linear regression

## Introduction
Gradient descent is a very commonly used optimization algorithm in modern machine learning. This two part series is intended to help people gain a better understanding of how it works by implementing it without the use of any machine learning libraries. In part one we will use gradient descent to solve a trivial linear regression problem to introduce the concepts. In part two we will build on these concepts to train a handwritten digit classifier. Feel free to skip straight to part two if you want. The format will be short explanations, followed by python code you can run from within this notebook.

## Simple Linear Regression
Simple linear regression is a statistical method that models the relationship between two variables. It attempts to predict as accurately as possible an independent variable (usually plotted on the y axis of a cartesian plane) from a dependent variable (usually plotted on the x axis). This relationship between dependent and independent variable is modelled by: 

$ y = mx + b $

where *y*, the output, is predicted by multiplying the input *x* by *m* and adding *b*. Optimal values for *m* and *b* are usually calculated using the ordinary least squares method. Instead we are going to use gradient descent to find (close to) optimal values for *m* and *b* to help further our understanding of gradient descent.


## Swedish Auto Insurance Dataset
The Swedish Auto Insurance Dataset involves predicting the total payment for all claims in thousands of Swedish Kronor, given the total number of claims. It was produced by the Swedish Committee on Analysis of Risk Premium in Motor Insurance. [More information](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/slr/frames/slr06.html). Lets load up the dataset and take a look at it:

In [ ]:
# import libraries
import numpy as np # python array library
import random
import matplotlib.pyplot as plt # python library for producing figures and plots
%matplotlib inline

# load dataset
dataset = np.genfromtxt('swedish_auto.csv', delimiter=',')
claims = dataset[:, 0] # number of claims (independent variable)
payments = dataset[:, 1] # total payment (dependent variable)

# plot dataset
fig_1 = plt.figure(1)
plt.scatter(payments, claims, color='blue')
plt.title('Swedish auto insurance dataset\nRaw data')
plt.xlabel('Number of claims')
plt.ylabel('Total payment for all claims (thousand Kronor)')
plt.show()

As you can see visually there is a clear pattern to the distribution of this data. Simple linear regression is likely to fit it well.

First we will quickly find an optimal solution for m and b using the ordinary least squares method to see what we are aiming to replicate with gradient descent. Feel free to skip over the details of the least squares method. If you are curious you can read more [here](https://en.wikipedia.org/wiki/Ordinary_least_squares).

In [ ]:
def ordinary_least_squares(x_set, y_set):
    
    numerator = sum(( (x - np.mean(x_set)) * (y - np.mean(y_set)) for x, y in zip(x_set, y_set) ))
    denominator = sum(( (x - np.mean(x_set))**2 for x in x_set ))

    m = numerator / denominator
    b = np.mean(y_set) - m * np.mean(x_set)

    return m, b

In [ ]:
m, b = ordinary_least_squares(claims, payments)
pred = [m * x + b for x in claims]

# plot
fig_2 = plt.figure(2)
plt.scatter(payments, claims, color='blue')
plt.plot(pred, claims, color='red')
plt.title('Swedish auto insurance dataset\nSimple linear regression')

plt.xlabel('Number of claims')
plt.ylabel('Total payment for all claims (thousand Kronor)')
plt.legend(['Least squares regression'])

plt.show()

print('m: {0}, b: {1}'.format(m, b))

As you can see our model does a pretty good job of predicting total payment from number of claims. Obviously statistical models are never going to perfectly predict real world phenomena (especially simple linear models like the one we are using here).

## Gradient Descent
Great, so using ordinary least squares we calculated the optimal solution for m and b to best fit the dataset with our *y = mx + b* model. Before we try and do the same using gradient descent, let's talk more about what gradient descent is and how it works.

Gradient descent is an optimization method. It is commonly used to optimize the values of variables in predictive models (in this example *m* and *b* from *y = mx + b*). It can be used for much more complex problems where there is not a nice easy formula to find an optimal solution (for example training neural networks).

The basic steps of gradient descent are as follows.
1. Start with random values for the variables you are trying to optimize
2. Compute some form of error measurement for your predictive model using know pairs of input/output
3. Use this error measurement to calculate a gradient for each variable
4. Change each variable based on its gradient
5. Repeat steps 2-4 until variables are (hopefully) optimized 

So what do I mean by gradient? Lets step back for a minute and talk about some calculus.

![y = f(x) graph](res/ax1.jpg)

Above is a graph of some arbitrary function *y = f(x)*. Let's say we are trying to find the value for *x* where *y* is minimum. Visually we can tell that it is around *x = 0*, and if we knew the function we could eaisly calculate the minimum mathematically. But what if we only knew a few points on the graph and the derivative of the function at those points? Could we still aproximate where y is minimum?

![p1 graph](res/ax2.jpg)

We can try! From the derivative of the function at point *p1*, we can tell that locally, reducing *x* will result in a reduction in *y*. In light of this new information let's say we take a new point *p2* that has a lower value of *x*.

![p2 graph](res/ax3.jpg)

Now we calculate the derivative at the point *p2*. And again we adjust our value of *x* based on the derivative. Lets say we keep following this strategy, we may eventually discover a value for $x$ that is very close to where *y* is minimum.

![optimization complete graph](res/ax4.jpg)

While this is a trivial example, it does illustrate the basic principle of how gradient descent optimization works. **Instead of optimizing x to find where y is minimum, gradient descent can also be used to optimize the variables of a predictive model to find where its 'mistakes' are minumum .** 'Mistakes' is usually expressed in terms of a **cost function**, which is some mathematical measure of a models performance chosen by the programmer. 

The strength of gradient descent is that this same method can still be applied to much more complex optimization problems with thousands of variables and very complex relationships between inputs and outputs (for example neural networks).

## Learning rate

We need to discuss another concept, and that is learning rate. The learning rate is a parameter that determines the magnitude of the change we make to the models variables based on the gradient (think of it as the distance between *p1* and *p2* in the graphs above). If the learning rate is too large, *m* and *b* will fluctuate wildly and our model will not converge towards an optimal solution. If learning rate is too small it will take far to long to converge towards an optimal solution.

![too high learning rate](res/ax5.jpg)

In the above graph the learning rate is too high, and the model does not converge towards an optimal solution

![too low learning rate](res/ax6.jpg)

In the above graph the learning rate is too low, and the model will take too long to converge towards an optimal solution

## Using gradient descent to solve our simple regression problem

Ok, let's see if we can use the strategy outlined above to solve our linear regression problem. That is to find values for *m* and *b* that result in our predictive model (*y = mx + b*) being as accurate as possible. These are the steps we will take:

1. Start with random values for m and b
2. For a random input/output pair in the swedish auto dataset, calculate how well the model did (cost function)
3. Calculate a gradient for m and a gradient for b with respect to the cost function
4. Update the values of m and b based on this gradient
5. Repeat steps 2-4

Before we can dive right into the code we need to go over a few things.

How exactly are we going to calculate a gradient for m and b based on how well the model performs?

First we have to define a cost function. This is what we are going to try and minimize. 

$ C(m,b) = (a - y)^2 $

Where *C* is our cost function, *m* and *b* are the variables we are trying to optimize, *a* is our models prediction and *y* is the ground truth value from our dataset.

Now we need to calculate a gradient for both *m* and *b* with respect to this cost function.

First lets expand the previous expression, substituting *mx + b* for *a*.

$ C(m,b) = (mx + b - y)^2 $

Now we want to compute a gradient for *m* and a gradient for *b* with respect to the cost function.

Partial derivative with respect to *m* (applying chain rule)

$ \frac {\partial C}{\partial m} = 2(mx + b - y) * x $

Partial derivative with respect to *b* (applying chain rule)

$ \frac {\partial C}{\partial b} = 2(mx + b - y)$

If you are having trouble understanding the maths, I recommend you review partial differential equations (PDE's) and the chain rule.

Now using these gradients we can determine how we want to change *m* and *b*. This change is multiplied by the learning rate. Because the change is multiplied by learning rate, we can ignore any constants (the leading 2 in this example).

$ \Delta m = (mx + b - y) * x * learning rate $

$ \Delta b = (mx + b - y) * learning rate $

Ok now that we have a plan of attack, lets implement it in code...


First we will create a few helper functions. Their purpose is explained in the comments

In [ ]:
# simple data generator, yields individual input/output pairs from the dataset
def simple_data_generator(x_dataset, y_dataset):
    
    assert x_dataset.shape[0] == y_dataset.shape[0]

    num_items = x_dataset.shape[0]
    index = 0

    while True:
        yield x_dataset[index], y_dataset[index]
        index += 1
        if index >= num_items:
            index = 0
            
# normalizes a dataset. Assumes numpy array
# normalization can have a range of meanings, in this context we are refering to feature scaling
# scaling variables to a common scale will help our model train better
def normalize_dataset(dataset, new_min, new_max):
      
    old_range = np.max(dataset) - np.min(dataset)
    new_range = new_max - new_min

    dataset *= (new_range / old_range)
    dataset += (new_min - np.min(dataset))

    return dataset

# calculates the mean error of our predictive model over the entire dataset
# this is not part of the training process, just evaluates the models performance
def mean_error(x_dataset, y_dataset, m, b):
    
    total_error = sum([abs(y - (m * x + b)) for x, y in zip(x_dataset, y_dataset)])
    return total_error / x_dataset.shape[0]

Next we set some things up in preparation for training

In [ ]:
learning_rate = 0.01 # choose a learning rate (how large steps to take at each training iteration)
num_iterations = 4000 # choose how many times we repeat steps 2 - 4
m = random.uniform(-1.0, 1.0) # initialize m and b to random values
b = random.uniform(-1.0, 1.0)

claims = normalize_dataset(claims, -1.0, 1.0) # scale both claims and payments to common scale (-1.0, 1.0)
payments = normalize_dataset(payments, -1.0, 1.0)
gen = simple_data_generator(claims, payments)

Use gradient descent to optimize values of m and b (i.e. train the model)

In [ ]:
for i in range(num_iterations):
    x, y = next(gen) # get input/output pair from our generator

    if i % 100 == 0:  # every 100 iterations print out an update on the training process
        me = mean_error(claims, payments, m, b)
        print('iteration: {0}, m: {1}, b: {2}, mean_error: {3}'.format(i, round(m, 4), round(b, 4), round(me, 4) ) )

    del_m = (m * x + b - y) * x * learning_rate  # calculate gradient for m relative to cost, multiply by learning rate
    del_b = (m * x + b - y) * learning_rate  # calculate gradient for b relative to cost, multiply by learning rate
    
    m -= del_m  # update m
    b -= del_b  # update b

Plot the results

In [ ]:
gd_pred = [m * x + b for x in claims]

fig_3 = plt.figure(3)
plt.scatter(payments, claims, color='blue')
plt.plot(gd_pred, claims, color='red')
plt.title('Swedish auto insurance dataset\nSimple linear regression')

plt.xlabel('Number of claims (normalized)')
plt.ylabel('Total payment for all claims (normalized)')
plt.legend(['Gradient descent'])

plt.show()

Not bad. We have found a (close to) optimal solution for our regression problem using gradient descent.

This same strategy is used to train most modern machine learning algorithms (scaled up and with some clever improvements, but at its heart the same concept). Now that you understand how gradient descent works in this simple context you are ready for part two, where we apply the same principles to train a neural network to classify handwritten digits.